In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
## import statements 
import importlib
import pandas as pd 
import matplotlib.pyplot as plt
import argparse
import warnings
warnings.filterwarnings('ignore')


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathon-sample-notebook/matrix_plot.jpg
/kaggle/input/hackathon-sample-notebook/__results__.html
/kaggle/input/hackathon-sample-notebook/loss_plot.jpg
/kaggle/input/hackathon-sample-notebook/__notebook__.ipynb
/kaggle/input/hackathon-sample-notebook/__output__.json
/kaggle/input/hackathon-sample-notebook/custom.css
/kaggle/input/hackathon-sample-notebook/project/genarate_plots.py
/kaggle/input/hackathon-sample-notebook/project/kaggle_utils.py
/kaggle/input/hackathon-sample-notebook/project/quora_dataset.py
/kaggle/input/hackathon-sample-notebook/project/README.md
/kaggle/input/hackathon-sample-notebook/project/main_functions.py
/kaggle/input/hackathon-sample-notebook/project/requirements.txt
/kaggle/input/hackathon-sample-notebook/project/__init__.py
/kaggle/input/hackathon-sample-notebook/project/models/t5_model.py
/kaggle/input/hackathon-sample-notebook/project/models/utils.py
/kaggle/input/hackathon-sample-notebook/project/models/__init__.py
/kaggle/input/hackathon-s

In [4]:
!git clone https://github.com/rahulgzb/project.git
!pip install -r project/requirements.txt

Cloning into 'project'...
remote: Enumerating objects: 225, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 225 (delta 43), reused 55 (delta 24), pack-reused 147
Receiving objects: 100% (225/225), 61.02 MiB | 24.61 MiB/s, done.
Resolving deltas: 100% (111/111), done.
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 9.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=dd5499b272faa97dac5984af9c874b2f6d17e85766e11ddaa56787b7dc6c77d0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [5]:
#reload_repo()
from project.kaggle_utils import reload_repo
from project.models import t5_model,utils
from project.main_functions import set_seed,args_dict,data_loading
from project.genarate_plots import save_plot
set_seed(42)

2024-07-29 17:01:38.559861: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 17:01:38.559988: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 17:01:38.677364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
## arrguments applicable for all functions 
args_dict.update({'data_dir': '/kaggle/working/project/data',
                  'output_dir': '/kaggle/working/project/result',
                  'num_train_epochs':5,
                  'max_seq_length': 256,
                  'train_batch_size':32,
                  'n_gpu':4,
                  'model_name_or_path':"google/flan-t5-small",
                  'tokenizer_name_or_path':'google/flan-t5-small',
                 })
args = argparse.Namespace(**args_dict)
print(args)
set_seed

Namespace(data_dir='/kaggle/working/project/data', output_dir='/kaggle/working/project/result', model_name_or_path='google/flan-t5-small', tokenizer_name_or_path='google/flan-t5-small', max_seq_length=256, learning_rate=0.0003, weight_decay=0.0, adam_epsilon=1e-08, warmup_steps=100, train_batch_size=32, num_train_epochs=5, gradient_accumulation_steps=32, n_gpu=4, fp_16=False, opt_level='O1', max_grad_norm=1.0, seed=42, eval_method='rougeL', eval_mode='max')


<function project.main_functions.set_seed(seed)>

In [25]:
model= t5_model.T5FineTuner(args)
tokenizer=model.tokenizer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
train_dataloader= data_loading(tokenizer,"train",args)
val_dataloader= data_loading(tokenizer,"val",args)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [9]:
helper=utils.trainner_helper(model,tokenizer)

### Block used for training the model for 5 epochs 

In [ ]:
utils.train_model(args,model,tokenizer,train_dataloader,val_dataloader)
save_plot(args)

### Evaluating the model after training for 5 epochs salecting the best model

In [34]:
checkpath_path='/kaggle/input/hackathon-sample-notebook/project/result/checkpoint_epoch_0.pt'
from project.models.t5_model import infer_single_sentence

In [35]:
import torch
model.load_state_dict(torch.load(checkpath_path))

<All keys matched successfully>

## fine tuned model for matrix 

In [24]:
import json
with open('/kaggle/input/hackathon-sample-notebook/project/result/model_scores_0.json','r') as f:
    matrix=json.load(f)
matrix

{'rouge1': 0.195,
 'rouge2': 0.0651,
 'rougeL': 0.1606,
 'bleu': 0.6578,
 'avg_val_loss': 3.4752628442852997,
 'gen_len': 30.5465,
 'avg_train_loss': 3.724306127141575}

In [44]:
sentence ="what is your name ?"

In [45]:
  
infer_single_sentence(model,tokenizer,sentence,)

"Mimik_human: I'm not a big fan of my name."